In [80]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import mapclassify as mc

Step0:  
Read Data and get basic county groups

0.0 Report Data

In [145]:
df_world_original = pd.read_csv("WHO-COVID-19-global-data.csv")
df_world_original

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-02-24T00:00:00Z,AF,Afghanistan,EMRO,1,1,0,0
1,2020-02-25T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
2,2020-02-26T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
3,2020-02-27T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
4,2020-02-28T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
...,...,...,...,...,...,...,...,...
18055,2020-05-24T00:00:00Z,ZW,Zimbabwe,AFRO,5,56,0,4
18056,2020-05-25T00:00:00Z,ZW,Zimbabwe,AFRO,0,56,0,4
18057,2020-05-26T00:00:00Z,ZW,Zimbabwe,AFRO,0,56,0,4
18058,2020-05-27T00:00:00Z,ZW,Zimbabwe,AFRO,0,56,0,4


In [82]:
df_world_original = df_world_original.drop(columns=['Country_code','WHO_region','New_cases','New_deaths'])
df_world_original

,Date_reported,Country,Cumulative_cases,Cumulative_deaths
0,2020-02-24T00:00:00Z,Afghanistan,1,0
1,2020-02-25T00:00:00Z,Afghanistan,1,0
2,2020-02-26T00:00:00Z,Afghanistan,1,0
3,2020-02-27T00:00:00Z,Afghanistan,1,0
4,2020-02-28T00:00:00Z,Afghanistan,1,0
...,...,...,...,...
18055,2020-05-24T00:00:00Z,Zimbabwe,56,4
18056,2020-05-25T00:00:00Z,Zimbabwe,56,4
18057,2020-05-26T00:00:00Z,Zimbabwe,56,4
18058,2020-05-27T00:00:00Z,Zimbabwe,56,4


In [83]:
df_world_original = df_world_original.rename(columns={"Date_reported": "date", "Country": "country", 
                                                      "Cumulative_cases": "cases", "Cumulative_deaths": "deaths"})
df_world_original

,date,country,cases,deaths
0,2020-02-24T00:00:00Z,Afghanistan,1,0
1,2020-02-25T00:00:00Z,Afghanistan,1,0
2,2020-02-26T00:00:00Z,Afghanistan,1,0
3,2020-02-27T00:00:00Z,Afghanistan,1,0
4,2020-02-28T00:00:00Z,Afghanistan,1,0
...,...,...,...,...
18055,2020-05-24T00:00:00Z,Zimbabwe,56,4
18056,2020-05-25T00:00:00Z,Zimbabwe,56,4
18057,2020-05-26T00:00:00Z,Zimbabwe,56,4
18058,2020-05-27T00:00:00Z,Zimbabwe,56,4


In [84]:
type(df_world_original.loc[0,'date'])

str

In [85]:
df_world_original['date'] = df_world_original['date'].str.strip('T00:00:00Z')
df_world_original

,date,country,cases,deaths
0,2020-02-24,Afghanistan,1,0
1,2020-02-25,Afghanistan,1,0
2,2020-02-26,Afghanistan,1,0
3,2020-02-27,Afghanistan,1,0
4,2020-02-28,Afghanistan,1,0
...,...,...,...,...
18055,2020-05-24,Zimbabwe,56,4
18056,2020-05-25,Zimbabwe,56,4
18057,2020-05-26,Zimbabwe,56,4
18058,2020-05-27,Zimbabwe,56,4


0.1 GeoJSON Data

In [87]:
old_world_geojson_df = gpd.read_file("World_Countries_Boundaries.geojson")
old_world_geojson_df.head(5)

,OBJECTID,CNTRY_NAME,geometry
0,1,Aruba,"POLYGON ((-69.88223 12.41111, -69.94695 12.436..."
1,2,Antigua and Barbuda,"MULTIPOLYGON (((-61.73889 17.54055, -61.81556 ..."
2,3,Afghanistan,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."
3,4,Algeria,"POLYGON ((-5.15213 30.18047, -5.08097 30.26236..."
4,5,Azerbaijan,"MULTIPOLYGON (((45.02583 41.03055, 44.98416 41..."


In [149]:
world_geojson_df = gpd.read_file("countries.geojson")
world_geojson_df

,ADMIN,ISO_A3,ISO_A2,geometry
0,Aruba,ABW,AW,"MULTIPOLYGON (((-69.99694 12.57758, -69.93639 ..."
1,Afghanistan,AFG,AF,"MULTIPOLYGON (((71.04980 38.40866, 71.05714 38..."
2,Angola,AGO,AO,"MULTIPOLYGON (((11.73752 -16.69258, 11.73851 -..."
3,Anguilla,AIA,AI,"MULTIPOLYGON (((-63.03767 18.21296, -63.09952 ..."
4,Albania,ALB,AL,"MULTIPOLYGON (((19.74777 42.57890, 19.74601 42..."
...,...,...,...,...
250,Samoa,WSM,WS,"MULTIPOLYGON (((-171.57002 -13.93816, -171.564..."
251,Yemen,YEM,YE,"MULTIPOLYGON (((53.30824 12.11839, 53.31027 12..."
252,South Africa,ZAF,ZA,"MULTIPOLYGON (((37.86378 -46.94085, 37.83644 -..."
253,Zambia,ZMB,ZM,"MULTIPOLYGON (((31.11984 -8.61663, 31.14102 -8..."


In [150]:
world_geojson_df['ISO_A2'].unique()

array(['AW', 'AF', 'AO', 'AI', 'AL', 'AX', 'AD', 'AE', 'AR', 'AM', 'AS',
       'AQ', 'TF', 'AG', 'AU', 'AT', 'AZ', 'BI', 'BE', 'BJ', 'BF', 'BD',
       'BG', 'BH', 'BS', 'BA', 'UM', 'BL', 'BY', 'BZ', 'BM', 'BO', 'BR',
       'BB', 'BN', 'BT', 'BW', 'CF', 'CA', 'CH', 'CL', 'CN', 'CI', '-',
       'CM', 'CD', 'CG', 'CK', 'CO', 'KM', 'CV', 'CR', 'CU', 'CW', 'KY',
       'CY', 'CZ', 'DE', 'DJ', 'DM', 'DK', 'DO', 'DZ', 'EC', 'EG', 'ER',
       'ES', 'EE', 'ET', 'FI', 'FJ', 'FK', 'FR', 'FO', 'FM', 'GA', 'GB',
       'GE', 'GG', 'GH', 'GI', 'GN', 'GM', 'GW', 'GQ', 'GR', 'GD', 'GL',
       'GT', 'GU', 'GY', 'HK', 'HM', 'HN', 'HR', 'HT', 'HU', 'ID', 'IM',
       'IN', 'IO', 'IE', 'IR', 'IQ', 'IS', 'IL', 'IT', 'JM', 'JE', 'JO',
       'JP', 'KZ', 'KE', 'KG', 'KH', 'KI', 'KN', 'KR', 'KW', 'LA', 'LB',
       'LR', 'LY', 'LC', 'LI', 'LK', 'LS', 'LT', 'LU', 'LV', 'MO', 'MF',
       'MA', 'MC', 'MD', 'MG', 'MV', 'MX', 'MH', 'MK', 'ML', 'MT', 'MM',
       'ME', 'MN', 'MP', 'MZ', 'MR', 'MS', 'MU', 'MW

In [151]:
len(world_geojson_df['ISO_A2'].unique())

239

In [152]:
list(set(df_world_original['Country_code'].unique())-set(world_geojson_df['ISO_A2'].unique()))

[nan,
 'YT',
 'BQ',
 'XK',
 'International commercial vessel',
 'RE',
 'GP',
 'MQ',
 'GF']

Group Data to world

In [88]:
pivot_world = pd.pivot_table(df_world_original , index=['country'], columns=['date'])
pivot_world = pivot_world.replace(np.nan, 0)
pivot_world

cases             \
date                                               2020-01-11 2020-01-12   
country                                                                    
Afghanistan                                               0.0        0.0   
Albania                                                   0.0        0.0   
Algeria                                                   0.0        0.0   
Andorra                                                   0.0        0.0   
Angola                                                    0.0        0.0   
...                                                       ...        ...   
Yemen                                                     0.0        0.0   
Zambia                                                    0.0        0.0   
Zimbabwe                                                  0.0        0.0   
occupied Palestinian territory, including east ...        0.0        0.0   
undefined                                                 0.0        0.0   

                                                                          \
date                                               2020-01-13 2020-01-14   
country                                                                    
Afghanistan                                               0.0        0.0   
Albania                                                   0.0        0.0   
Algeria                                                   0.0        0.0   
Andorra                                                   0.0        0.0   
Angola                                                    0.0        0.0   
...                                                       ...        ...   
Yemen                                                     0.0        0.0   
Zambia                                                    0.0        0.0   
Zimbabwe                                                  0.0        0.0   
occupied Palestinian territory, including east ...        0.0        0.0   
undefined                                                 0.0        0.0   

                                                                          \
date                                               2020-01-15 2020-01-16   
country                                                                    
Afghanistan                                               0.0        0.0   
Albania                                                   0.0        0.0   
Algeria                                                   0.0        0.0   
Andorra                                                   0.0        0.0   
Angola                                                    0.0        0.0   
...                                                       ...        ...   
Yemen                                                     0.0        0.0   
Zambia                                                    0.0        0.0   
Zimbabwe                                                  0.0        0.0   
occupied Palestinian territory, including east ...        0.0        0.0   
undefined                                                 0.0        0.0   

                                                                          \
date                                               2020-01-17 2020-01-18   
country                                                                    
Afghanistan                                               0.0        0.0   
Albania                                                   0.0        0.0   
Algeria                                                   0.0        0.0   
Andorra                                                   0.0        0.0   
Angola                                                    0.0        0.0   
...                                                       ...        ...   
Yemen                                                     0.0        0.0   
Zambia                                                    0.0        0.0   
Zimbabwe                 

0.3 Get Date Info

In [89]:
date= pivot_world['cases'].columns
date

Index(['2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15',
       '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-2',
       ...
       '2020-05-19', '2020-05-2', '2020-05-21', '2020-05-22', '2020-05-23',
       '2020-05-24', '2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28'],
      dtype='object', name='date', length=139)

In [90]:
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [91]:
dt_start = datetime.strptime(dt_str_start, "%Y-%m-%d")
dt_end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=dt_start,end=dt_end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

139 DatetimeIndex(['2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14',
               '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18',
               '2020-01-19', '2020-01-20',
               ...
               '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22',
               '2020-05-23', '2020-05-24', '2020-05-25', '2020-05-26',
               '2020-05-27', '2020-05-28'],
              dtype='datetime64[ns]', length=139, freq='D')
139 ['2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-

Step1:  
Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [92]:
# import json
# pivot_world['cases_ts'] = json.dumps({"values": pivot_world['cases'].values.tolist()[0]})
# pivot_world['deaths_ts'] =  json.dumps({"values": pivot_world['deaths'].values.tolist()[0]})

pivot_world['cases_ts'] =  pivot_world['cases'].values.tolist()
pivot_world['deaths_ts'] =  pivot_world['deaths'].values.tolist()

Form Dynamic classes for new_case and new_death 

In [93]:
new_case_dict = {}
for i in range(1,len(date)):
    q6 = mc.Quantiles(pivot_world['cases_ts'].apply(lambda x: x[i] - x[i - 1]).values, k=6)
    new_case_dict[i] = ','.join(q6.bins.astype(int).astype(str))

C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:237: UserWarning: Warning: Not enough unique values in array to form k classes
  "Warning: Not enough unique values in array to form k classes", UserWarning
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 1
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:890: RuntimeWarning: invalid value encountered in double_scalars
  gadf = 1 - self.adcm / adam
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 2
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaconda3\envs\wherecovid19\lib\site-packages\mapclassify\classifiers.py:239: UserWarning: Warning: setting k to 3
  Warn("Warning: setting k to %d" % k_q, UserWarning)
C:\Users\lymin\Anaco

In [94]:
dynamic_cl_dict = {}
dynamic_cl_dict['county'] = {'new_case':new_case_dict, 'new_death':{}}
dynamic_cl_dict['state'] = {}

In [95]:
import json
with open('dynamic_classes.json','w') as json_file:
    json.dump(dynamic_cl_dict, json_file)

In [96]:
pivot_world['dt_first_case'] = (pivot_world['cases'] > 0).idxmax(axis=1)
pivot_world['dt_first_death'] = (pivot_world['deaths'] > 0).idxmax(axis=1)
#For death reports, deal with no deaths world
pivot_world.loc[pivot_world['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_world.head(5)

cases                                                         \
date        2020-01-11 2020-01-12 2020-01-13 2020-01-14 2020-01-15 2020-01-16   
country                                                                         
Afghanistan        0.0        0.0        0.0        0.0        0.0        0.0   
Albania            0.0        0.0        0.0        0.0        0.0        0.0   
Algeria            0.0        0.0        0.0        0.0        0.0        0.0   
Andorra            0.0        0.0        0.0        0.0        0.0        0.0   
Angola             0.0        0.0        0.0        0.0        0.0        0.0   

                                                        ...     deaths  \
date        2020-01-17 2020-01-18 2020-01-19 2020-01-2  ... 2020-05-23   
country                                                 ...              
Afghanistan        0.0        0.0        0.0       0.0  ...      211.0   
Albania            0.0        0.0        0.0       0.0  ...       31.0   
Algeria            0.0        0.0        0.0       0.0  ...      582.0   
Andorra            0.0        0.0        0.0       0.0  ...       51.0   
Angola             0.0        0.0        0.0       0.0  ...        3.0   

                                                                    \
date        2020-05-24 2020-05-25 2020-05-26 2020-05-27 2020-05-28   
country                                                              
Afghanistan      218.0      219.0      220.0      226.0      226.0   
Albania           32.0       32.0       33.0       33.0       33.0   
Algeria          592.0      600.0      609.0      617.0      623.0   
Andorra           51.0       51.0       51.0       51.0       51.0   
Angola             3.0        4.0        4.0        4.0        4.0   

                                                      cases_ts  \
date                                                             
country                                                          
Afghanistan  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
Albania      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
Algeria      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
Andorra      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
Angola       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                     deaths_ts dt_first_case  \
date                                                                           
country                                                                        
Afghanistan  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-02-24   
Albania      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-03-09   
Algeria      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-02-25   
Andorra      [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-03-02   
Angola       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...    2020-03-21   

            dt_first_death  
date                        
country                     
Afghanistan     2020-03-23  
Albania         2020-03-13  
Algeria         2020-03-12  
Andorra         2020-03-24  
Angola          2020-03-31  

[5 rows x 282 columns]

Step2:  
Add today_case, today_new_case, today_death, today_new_death

In [97]:
yesterday = date.values[-2]
pivot_world['today_case'] = pivot_world['cases'][dt_str_end]
pivot_world['today_new_case'] = pivot_world['cases'][dt_str_end] - pivot_world['cases'][yesterday]
pivot_world['today_death'] = pivot_world['deaths'][dt_str_end]
pivot_world['today_new_death'] = pivot_world['deaths'][dt_str_end] - pivot_world['deaths'][yesterday]

Step3:  
Extract necessary columns

In [98]:
report_df = pivot_world[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['country','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,country,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Afghanistan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-02-24,2020-03-23,12379.0,0.0,226.0,0.0
1,Albania,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-09,2020-03-13,1050.0,0.0,33.0,0.0
2,Algeria,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-02-25,2020-03-12,8857.0,160.0,623.0,6.0
3,Andorra,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-02,2020-03-24,763.0,0.0,51.0,0.0
4,Angola,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-21,2020-03-31,71.0,0.0,4.0,0.0


Step4:  
Assign Geometries to world

In [126]:
final_df = pd.merge(old_world_geojson_df, report_df, how='left', left_on=['CNTRY_NAME'], right_on = ['country'])
final_df.columns

Index(['OBJECTID', 'CNTRY_NAME', 'geometry', 'country', 'cases_ts',
       'deaths_ts', 'dt_first_case', 'dt_first_death', 'today_case',
       'today_new_case', 'today_death', 'today_new_death'],
      dtype='object')

In [127]:
final_df

,OBJECTID,CNTRY_NAME,geometry,country,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,1,Aruba,"POLYGON ((-69.88223 12.41111, -69.94695 12.436...",Aruba,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-17,2020-04-16,101.0,0.0,3.0,0.0
1,2,Antigua and Barbuda,"MULTIPOLYGON (((-61.73889 17.54055, -61.81556 ...",Antigua and Barbuda,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-13,2020-04-09,25.0,0.0,3.0,0.0
2,3,Afghanistan,"POLYGON ((61.27656 35.60725, 61.29638 35.62853...",Afghanistan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-02-24,2020-03-23,12379.0,0.0,226.0,0.0
3,4,Algeria,"POLYGON ((-5.15213 30.18047, -5.08097 30.26236...",Algeria,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-02-25,2020-03-12,8857.0,160.0,623.0,6.0
4,5,Azerbaijan,"MULTIPOLYGON (((45.02583 41.03055, 44.98416 41...",Azerbaijan,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-02-29,2020-03-18,4403.0,0.0,52.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
246,247,Western Samoa,"MULTIPOLYGON (((-172.59650 -13.50911, -172.360...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,248,Swaziland,"POLYGON ((30.89944 -26.77195, 30.88028 -26.793...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,249,Yemen,"MULTIPOLYGON (((48.68639 14.03750, 48.55388 14...",Yemen,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-04-1,2020-05-01,253.0,0.0,50.0,0.0
249,250,Zambia,"POLYGON ((30.21302 -14.98172, 30.21917 -15.096...",Zambia,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-18,2020-04-03,1057.0,137.0,7.0,0.0


In [131]:
final_df['country'].isna().value_counts()

False    173
True      78
Name: country, dtype: int64

In [146]:
df_world_original['Country_code'].unique()

array(['AF', 'AL', 'DZ', 'AD', 'AO', 'AI', 'AG', 'AR', 'AM', 'AW', 'AU',
       'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM',
       'BT', 'BO', 'BQ', 'BA', 'BW', 'BR', 'VG', 'BN', 'BG', 'BF', 'BI',
       'CV', 'KH', 'CM', 'CA', 'KY', 'CF', 'TD', 'CL', 'CN', 'CO', 'KM',
       'CG', 'CR', 'CI', 'HR', 'CU', 'CW', 'CY', 'CZ', 'CD', 'DK', 'DJ',
       'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE', 'SZ', 'ET', 'FK',
       'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'GA', 'GM', 'GE', 'DE', 'GH',
       'GI', 'GR', 'GL', 'GD', 'GP', 'GU', 'GT', 'GG', 'GN', 'GW', 'GY',
       'HT', 'VA', 'HN', 'HU', 'IS', 'IN', 'ID',
       'International commercial vessel', nan, 'IR', 'IQ', 'IE', 'IM',
       'IL', 'IT', 'JM', 'JP', 'JE', 'JO', 'KZ', 'KE', 'XK', 'KW', 'KG',
       'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU', 'MG', 'MW',
       'MY', 'MV', 'ML', 'MT', 'MQ', 'MR', 'MU', 'YT', 'MX', 'MC', 'MN',
       'ME', 'MS', 'MA', 'MZ', 'MM', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE',
    

In [148]:
len(df_world_original['Country_code'].unique())

216

In [139]:
old_world_geojson_df['CNTRY_NAME'].unique()

array(['Aruba', 'Antigua and Barbuda', 'Afghanistan', 'Algeria',
       'Azerbaijan', 'Albania', 'Armenia', 'Andorra', 'Angola',
       'American Samoa', 'Argentina', 'Australia', 'Austria', 'Anguilla',
       'Antarctica', 'Bahrain', 'Barbados', 'Botswana', 'Bermuda',
       'Belgium', 'Bahamas, The', 'Bangladesh', 'Belize',
       'Bosnia and Herzegovina', 'Bolivia', 'Myanmar (Burma)', 'Benin',
       'Byelarus', 'Solomon Islands', 'Brazil', 'Bhutan', 'Bulgaria',
       'Bouvet Island', 'Brunei', 'Burundi', 'Canada', 'Cambodia', 'Chad',
       'Sri Lanka', 'Congo', 'Zaire', 'China', 'Chile', 'Cayman Islands',
       'Cocos (Keeling) Islands', 'Cameroon', 'Comoros', 'Colombia',
       'Northern Mariana Islands', 'Costa Rica',
       'Central African Republic', 'Cuba', 'Cape Verde', 'Cook Islands',
       'Cyprus', 'Denmark', 'Djibouti', 'Dominica', 'Jarvis Island',
       'Dominican Republic', 'Ecuador', 'Egypt', 'Ireland',
       'Equatorial Guinea', 'Estonia', 'Eritrea', 'El Salvado

In [141]:
print(sorted(old_world_geojson_df['CNTRY_NAME'].unique()))

['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain', 'Baker Island', 'Bangladesh', 'Barbados', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Byelarus', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Islas Malvinas)', 'Faroe Islands', 'Federate

["Lao People's Democratic Republic",
 'Saint Kitts and Nevis',
 'Sint Maarten',
 'International conveyance (Diamond Princess)',
 'Saint Martin',
 'Bahamas',
 'Democratic Republic of the Congo',
 'Cabo Verde',
 'Republic of Korea',
 'Curacao',
 'Russian Federation',
 'The United Kingdom',
 'Bolivia (Plurinational State of)',
 'North Macedonia',
 'Isle of Man',
 'Saint Vincent and the Grenadines',
 'Côte d’Ivoire',
 'Gambia',
 'Northern Mariana Islands (Commonwealth of the)',
 'Holy See',
 'Saint Pierre and Miquelon',
 'Myanmar',
 'Eswatini',
 'Bonaire, Sint Eustatius and Saba',
 'United States Virgin Islands',
 'Belarus',
 'South Sudan',
 'Saint Barthélemy',
 'Brunei Darussalam',
 'Iran (Islamic Republic of)',
 'United Republic of Tanzania',
 'Réunion',
 'Syrian Arab Republic',
 'Saint Lucia',
 'Czechia',
 'Viet Nam',
 'Timor-Leste',
 'Falkland Islands (Malvinas) [2]',
 'Kosovo[1]',
 'undefined',
 'occupied Palestinian territory, including east Jerusalem',
 'Republic of Moldova',
 'Vene

In [ ]:
df_world_original.replace({'country' : {"Lao People's Democratic Republic" : 'Laos', 
                                        'Saint Kitts and Nevis' : 'St. Kitts and Nevis', 
                                        'Bahamas' : 'Bahamas, The',
                                        'Democratic Republic of the Congo' : 'Congo',
                                         'Cabo Verde' : 'Cape Verde',
                                         'Republic of Korea' : 'North Korea',
                                         'Russian Federation' : 'Russia',
                                         'The United Kingdom' : 'United Kingdom',
                                         'Bolivia (Plurinational State of)' : 'Bolivia',
                                         'North Macedonia' : 'Macedonia',
                                         'Isle of Man' : 'Man, Isle of',
                                         'Saint Vincent and the Grenadines' : 'St. Vincent and the Grenadines',
                                         'Côte d’Ivoire' : 'Ivory Coast',
                                         'Gambia' : 'Gambia, The',
                                         'Northern Mariana Islands (Commonwealth of the)' : 'Northern Mariana Islands',
                                         'Saint Pierre and Miquelon' : 'St. Pierre and Miquelon',
                                         'Myanmar' : 'Myanmar (Burma)',
                                         'Eswatini' : 'Swaziland',
                                         'United States Virgin Islands' : 'Virgin Islands',
                                         'Belarus' : 'Byelarus',
                                         'Brunei Darussalam' : 'Brunei',
                                         'Iran (Islamic Republic of)' : 'Iran',
                                         'United Republic of Tanzania' : 'Tanzania, United Republic of',
                                         'Réunion' : 'Reunion',
                                         'Syrian Arab Republic' : 'Syria',
                                         'Saint Lucia' : 'St. Lucia',
                                         'Czechia' : 'Czech Republic',
                                         'Viet Nam' : 'Vietnam',
                                         'Timor-Leste' : ,
                                         'Falkland Islands (Malvinas) [2]',
                                         'Kosovo[1]',
                                         'undefined',
                                         'occupied Palestinian territory, including east Jerusalem',
                                         'Republic of Moldova',
                                         'Venezuela (Bolivarian Republic of)',
                                         'United States of America'
                                       }})

Step5:  
Deal with nan values (world not shows up in geojson) (More works)

In [101]:
final_df[['today_case','today_new_case','today_death','today_new_death']] = final_df[['today_case','today_new_case','today_death','today_new_death']].replace(np.nan,0)
final_df['country'] = final_df['country'].replace(np.nan, final_df['CNTRY_NAME'])

In [102]:
template =final_df[~final_df['cases_ts'].isna()]['cases_ts']
template = template.iloc[0]

In [103]:
for x in range(0,len(template)):
    template[x] = 0
template = ','.join(map(str, template))

In [104]:
final_df['cases_ts'] = final_df['cases_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df['deaths_ts'] = final_df['deaths_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df.head(5)

,OBJECTID,CNTRY_NAME,geometry,country,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,1,Aruba,"POLYGON ((-69.88223 12.41111, -69.94695 12.436...",Aruba,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-17,2020-04-16,101.0,0.0,3.0,0.0
1,2,Antigua and Barbuda,"MULTIPOLYGON (((-61.73889 17.54055, -61.81556 ...",Antigua and Barbuda,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-13,2020-04-09,25.0,0.0,3.0,0.0
2,3,Afghanistan,"POLYGON ((61.27656 35.60725, 61.29638 35.62853...",Afghanistan,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-02-24,2020-03-23,12379.0,0.0,226.0,0.0
3,4,Algeria,"POLYGON ((-5.15213 30.18047, -5.08097 30.26236...",Algeria,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-02-25,2020-03-12,8857.0,160.0,623.0,6.0
4,5,Azerbaijan,"MULTIPOLYGON (((45.02583 41.03055, 44.98416 41...",Azerbaijan,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-02-29,2020-03-18,4403.0,0.0,52.0,0.0


In [105]:
final_df

,OBJECTID,CNTRY_NAME,geometry,country,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,1,Aruba,"POLYGON ((-69.88223 12.41111, -69.94695 12.436...",Aruba,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-17,2020-04-16,101.0,0.0,3.0,0.0
1,2,Antigua and Barbuda,"MULTIPOLYGON (((-61.73889 17.54055, -61.81556 ...",Antigua and Barbuda,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-13,2020-04-09,25.0,0.0,3.0,0.0
2,3,Afghanistan,"POLYGON ((61.27656 35.60725, 61.29638 35.62853...",Afghanistan,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-02-24,2020-03-23,12379.0,0.0,226.0,0.0
3,4,Algeria,"POLYGON ((-5.15213 30.18047, -5.08097 30.26236...",Algeria,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-02-25,2020-03-12,8857.0,160.0,623.0,6.0
4,5,Azerbaijan,"MULTIPOLYGON (((45.02583 41.03055, 44.98416 41...",Azerbaijan,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-02-29,2020-03-18,4403.0,0.0,52.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
246,247,Western Samoa,"MULTIPOLYGON (((-172.59650 -13.50911, -172.360...",Western Samoa,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
247,248,Swaziland,"POLYGON ((30.89944 -26.77195, 30.88028 -26.793...",Swaziland,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",NaN,NaN,0.0,0.0,0.0,0.0
248,249,Yemen,"MULTIPOLYGON (((48.68639 14.03750, 48.55388 14...",Yemen,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-1,2020-05-01,253.0,0.0,50.0,0.0
249,250,Zambia,"POLYGON ((30.21302 -14.98172, 30.21917 -15.096...",Zambia,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-18,2020-04-03,1057.0,137.0,7.0,0.0


Step6:  
Finalize the Dataframe

In [106]:
final_df['dt_start'] = dt_str_start
final_df['dt_end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df['population'] = 0
final_df.head(1)

,OBJECTID,CNTRY_NAME,geometry,country,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death,dt_start,dt_end,dt_unit,population
0,1,Aruba,"POLYGON ((-69.88223 12.41111, -69.94695 12.436...",Aruba,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-17,2020-04-16,101.0,0.0,3.0,0.0,2020-01-11,2020-05-28,day,0


Step 7:  
Output file

In [107]:
final_df.to_file("who_world_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done


New Data Structure (Full version)

In [108]:
# pivot_world.columns = ['_'.join(col).strip() for col in pivot_world.columns.values]
# report_df_2 = pivot_world.reset_index()

In [109]:
# report_df_2.head(5)

In [110]:
# pivot_world.head(5)

In [111]:
# final_df_2 = pd.merge(old_world_geojson_df[["NAME", "state_name", "population", "geometry"]], report_df_2, how='right', left_on=['NAME','state_name'], right_on = ['county','state'])
# final_df_2 = final_df_2.drop(['state','county'], axis = 1)
# final_df_2['dt_start'] = dt_start
# final_df_2['dt_end'] = dt_end
# final_df_2.head(5)

In [112]:
# final_df_2.to_file(r"./nyt_world_data_full.geojson", driver='GeoJSON', encoding='utf-8')
# print("done")

Generate JSON file

In [113]:
# county_list = []
# for state in df_world_original.state.unique():
#     for county in df_world_original.loc[(df_world_original['state'] == state)].county.unique():
#         county_list.append({"name": county, "state": state})
 
# print(len(county_list), county_list)


In [114]:
# data_df = report_df.set_index(report_df["county"]+", "+report_df["state"])
# data_dict =data_df.to_dict(orient='index')

In [115]:
# df = df_world_original.groupby(['county', 'state'], sort=False)[['cases', 'deaths']].max().sort_values("cases", ascending=False)
# county_state_list = list(map(lambda x: x[0]+", "+x[1], df.index.tolist()))
# meta = {"dates": dt_range_str,"states": county_state_list, "cases": df["cases"].values.tolist(), "deaths": df["deaths"].values.tolist()}
# data_dict["metadata"] = meta

In [116]:
# data_dict

In [117]:
# import json
# with open('./nyt_world_data.json', 'w') as outfile:
#     json.dump(data_dict, outfile)